In [ ]:
import sys

sys.path.insert(0, '/mnt/data/molchanov/dltranz')

import torch
import torch.nn.functional as F

import matplotlib.pyplot as plt

from domyshnik.models import *
from domyshnik.data import *
from domyshnik.constants import *
from domyshnik.utils import *

def draw(imgs):
        if isinstance(imgs, list):
            imgs = torch.stack(imgs)
        fig = plt.figure()
        rows, columns = 1, imgs.shape[0]
        for i in range(imgs.shape[0]):
            fig.add_subplot(rows, columns, i+1)
            if imgs[i].size(0) == 3:
                plt.imshow(imgs[i].transpose(0, 1).transpose(1, 2))
            else:
                plt.imshow(imgs[i])
        plt.show()
        
%matplotlib inline


In [ ]:
def cifar_torch_augmentation_(p=1):
    return torchvision.transforms.Compose([
                
        transforms.RandomApply([
            transforms.RandomChoice([
                transforms.RandomAffine(degrees=45, 
                                        translate=(0.2, 0.2), 
                                        scale=(0.7, 1.3), 
                                        shear=10, 
                                        resample=False),
                transforms.RandomResizedCrop(size=32, 
                                             scale=(0.6, 1.5), 
                                             ratio=(0.75, 1.3), 
                                             interpolation=2)]
            )], p=0.5
        ),
        
        transforms.RandomApply([
            transforms.ColorJitter(brightness=0.1, 
                                   contrast=0.7, 
                                   saturation=(0.5, 1), 
                                   hue=0.2)], p=0.5
        ),
        
        transforms.RandomApply([
            transforms.RandomVerticalFlip(),
            transforms.RandomHorizontalFlip()], p=0.5
        ),
        
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
    ])

def get_cifar10_test_loader_(batch_size, n_augments=4, augment_labels=False):
    data_test = MetrLearnDataset(dataset=cifar10_test_dataset(), 
                            augmenter=cifar_torch_augmentation_(p=1), 
                            n_augments=n_augments,
                            augment_labels=augment_labels)
        
    test_data_loader = torch.utils.data.DataLoader(data_test,
                                              batch_size=batch_size,
                                              shuffle=False,
                                              num_workers=5)
    return test_data_loader

loader = get_cifar10_test_loader_(6, n_augments=3)
sample = next(iter(loader))[0]
print(sample.size(), sample[0].size())
draw(sample[1])

In [ ]:
loader = get_cifar10_test_loader(BATCH_SIZE, n_augments=3)
sample = next(iter(loader))[0]
print(sample.size(), sample[0].size())
draw(sample[0])